In [101]:
import os
from socketserver import BaseRequestHandler
from typing import Any

from dotenv import load_dotenv
from fastavro import json_reader
from vertexai.preview.reasoning_engines.templates.langchain import BaseTool
from win32comext.shell.demos.servers.folder_view import tasks
from zmq.ssh.tunnel import select_random_ports

load_dotenv()
demo=os.getenv('FIN_DATA_API')

ImportError: cannot import name 'BaseTool' from 'vertexai.preview.reasoning_engines.templates.langchain' (C:\Users\karthik cse\AppData\Local\Programs\Python\Python311\Lib\site-packages\vertexai\preview\reasoning_engines\templates\langchain.py)

In [102]:
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = f'https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol=HAL&apikey={demo}'
r = requests.get(url)
data = r.json()

print(data)

{'Meta Data': {'1. Information': 'Weekly Prices (open, high, low, close) and Volumes', '2. Symbol': 'HAL', '3. Last Refreshed': '2025-02-26', '4. Time Zone': 'US/Eastern'}, 'Weekly Time Series': {'2025-02-26': {'1. open': '26.3000', '2. high': '27.1400', '3. low': '25.7350', '4. close': '25.9200', '5. volume': '30780652'}, '2025-02-21': {'1. open': '26.2200', '2. high': '27.3850', '3. low': '26.0950', '4. close': '26.2600', '5. volume': '39231993'}, '2025-02-14': {'1. open': '25.5500', '2. high': '26.7800', '3. low': '25.5000', '4. close': '26.1600', '5. volume': '41065592'}, '2025-02-07': {'1. open': '25.6900', '2. high': '26.5400', '3. low': '25.1600', '4. close': '25.1900', '5. volume': '50229041'}, '2025-01-31': {'1. open': '27.8800', '2. high': '28.1000', '3. low': '25.8200', '4. close': '26.0200', '5. volume': '65014046'}, '2025-01-24': {'1. open': '30.2000', '2. high': '30.2600', '3. low': '27.4700', '4. close': '27.8000', '5. volume': '62369091'}, '2025-01-17': {'1. open': '27.

In [103]:
import json
print(json.dumps(data,indent=4))

{
    "Meta Data": {
        "1. Information": "Weekly Prices (open, high, low, close) and Volumes",
        "2. Symbol": "HAL",
        "3. Last Refreshed": "2025-02-26",
        "4. Time Zone": "US/Eastern"
    },
    "Weekly Time Series": {
        "2025-02-26": {
            "1. open": "26.3000",
            "2. high": "27.1400",
            "3. low": "25.7350",
            "4. close": "25.9200",
            "5. volume": "30780652"
        },
        "2025-02-21": {
            "1. open": "26.2200",
            "2. high": "27.3850",
            "3. low": "26.0950",
            "4. close": "26.2600",
            "5. volume": "39231993"
        },
        "2025-02-14": {
            "1. open": "25.5500",
            "2. high": "26.7800",
            "3. low": "25.5000",
            "4. close": "26.1600",
            "5. volume": "41065592"
        },
        "2025-02-07": {
            "1. open": "25.6900",
            "2. high": "26.5400",
            "3. low": "25.1600",
          

In [104]:
from crewai import Crew,LLM,Task,Agent,Process
googleapi=os.getenv('GEMINI_API_KEY')
llm=LLM(
    model='gemini/gemini-2.0-flash-exp',
    api_key=googleapi
)

In [105]:

import requests
import json
from crewai_tools import BaseTool

class APIRequestTool(BaseTool):
    name: str = "API tool"
    description: str = "Makes HTTP requests to the given URL"
    url: str

    def _run(self):
        return requests.get(self.url).json()  # Use self.url in the request


In [106]:
symb='HAL'

In [107]:
scrape_tool1=APIRequestTool(url=f'https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol={symb}&apikey={demo}')

In [108]:

scrape_tool2=APIRequestTool(url=f'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol={symb}&apikey={demo}')

In [109]:
weekly_Agent=Agent(
    role="To summarize the weekly data that is scraped from your tool",
    goal="to create fine report on the weekly stock data in the markdown format about the meta data and numerical data that you get",
    backstory="You are one of the best agents in the stock market who generates the report after reviewing  the data and company",
    llm=llm,
    tools=[scrape_tool1]
)

In [110]:


monthly_agent=Agent(
    role="To summarize the monthly data that is scraped from your tool",
    goal="to create fine report on the monthly stock data in the markdown format about the meta data and numerical data that you get",
    backstory="You are one of the best agents in the stock market who generates the report after reviewing  the data and company",
    llm=llm,
    tools=[scrape_tool2]
)

In [111]:
summarizer=Agent(
    role="To give useful data oriented insights about the company to the user",
    goal="To summarize the strengths and weakness of the particular stock based on weekly and monthly the reports that you get from two agents",
    backstory="You are one of the best agents in the stock market who generates the report and provide insights after reviewing  the data and company",
    llm=llm,
)

In [112]:
weekly_task=Task(
    description="conduct the walkthrough of the report and review",
    expected_output="Well structured markdown format data",
    agent=weekly_Agent
)
monthly_task=Task(
     description="conduct the walkthrough of the report and review",
    expected_output="Well structured markdown format data",
    agent=monthly_agent
)
summary_task=Task(
     description="conduct the walkthrough of the weekly report and monthly report provided by both agents and generate summary with insights",
    expected_output="Well structured markdown format data",
    agent=summarizer,
    context=[weekly_task,monthly_task]
)

In [113]:
crew=Crew(
    agents=[weekly_Agent,monthly_agent,summarizer],
    tasks=[weekly_task,monthly_task,summary_task],
    process= Process.sequential,
            verbose=True,
)

2025-02-27 16:59:14,299 - 8664 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [114]:
res=crew.kickoff()

# Agent: To summarize the weekly data that is scraped from your tool
## Task: conduct the walkthrough of the report and review


# Agent: To summarize the weekly data that is scraped from your tool
## Thought: Okay, I understand. I need to create a well-structured markdown report summarizing the weekly stock data I receive from the API tool. I will start by calling the API tool and then format the data into a comprehensive report.
## Using tool: API tool
## Tool Input: 
"{}"
## Tool Output: 
{'Meta Data': {'1. Information': 'Weekly Prices (open, high, low, close) and Volumes', '2. Symbol': 'HAL', '3. Last Refreshed': '2025-02-26', '4. Time Zone': 'US/Eastern'}, 'Weekly Time Series': {'2025-02-26': {'1. open': '26.3000', '2. high': '27.1400', '3. low': '25.7350', '4. close': '25.9200', '5. volume': '30780652'}, '2025-02-21': {'1. open': '26.2200', '2. high': '27.3850', '3. low': '26.0950', '4. close': '26.2600', '5. volume': '39231993'}, '2025-02-14': {'1. open': '25.5500', '2. high': '

In [115]:

from IPython import display
display.Markdown(res.raw)

```markdown
## HAL Stock Analysis: Weekly and Monthly Report Summary

**Data Overview:**

*   **Symbol:** HAL
*   **Last Refreshed:**
    *   Weekly: 2025-02-26
    *   Monthly: 2025-02-28
*   **Time Zone:** US/Eastern

**I. Weekly Performance Analysis:**

*   **Recent Trend (February 2025):**  The stock has shown a slight downward trend in February. Opening at 26.30 on 2025-02-26 and closing at 25.92. It experienced a volatile week with a high of 27.14 and a low of 25.735. Previous weeks in February show similar volatility.
*   **January 2025:**  Started high at 30.20 and closed the month lower at 26.02, indicating a declining trend throughout January.  High trading volumes were observed, especially in the latter half of the month.
*   **December 2024:** HAL experienced considerable fluctuation. It began the month at 31.93 and ended at 26.79. A significant drop occurred mid-month (2024-12-20), with the price falling to 25.97 amid high volume.
*   **Late 2024 (Nov-Oct):**  Showed more stability and growth, rising from 27.42 (2024-11-01) to a peak around 32.57 before declining towards the end of November.

**II. Monthly Performance Analysis:**

*   **February 2025:** HAL's monthly performance shows volatility, with a wide range between high and low prices (27.3850 and 25.1600 respectively), and closes lower than it opens, indicating negative momentum.
*   **January 2025:** Similar to the weekly data, the monthly data shows a declining trend from the start to the end of the month.
*   **2024 Overview:**
    *   **Q4 (Oct-Dec):** Started with a moderate correction in October, followed by a recovery in November, and a decline in December.
    *   **Q3 (Jul-Sep):** Demonstrated a strong recovery, particularly in September, with a substantial increase in price and volume.
    *   **Q2 (Apr-Jun):** Showed a general downtrend from April to June.
    *   **Q1 (Jan-Mar):** Started strong in January but experienced a decline through February and March.

**III. Volume Analysis:**

*   **Weekly:** Volumes tend to be higher during weeks with significant price changes, suggesting increased trading activity during periods of uncertainty or key price levels.
*   **Monthly:** Consistently high trading volumes throughout the year, indicating strong investor interest in HAL.  September 2024 recorded the highest volume, coinciding with a significant price increase.

**IV. Insights & Summary:**

*   **Strengths:**
    *   **High Trading Volume:**  Consistent high trading volume suggests strong liquidity and investor interest, making it easier to enter and exit positions.
    *   **Recovery Potential:** Demonstrated ability to recover from downturns, as seen in September 2024.
*   **Weaknesses:**
    *   **Price Volatility:** Significant price fluctuations on both weekly and monthly scales, which could indicate higher risk.
    *   **Recent Downtrend:** The stock has been on a downtrend in early 2025, which may concern investors.
*   **Overall Assessment:**

HAL exhibits characteristics of a volatile stock with potential for significant gains but also substantial risk. The high trading volume and past recovery trends are positive signs. However, the recent downtrend and price volatility suggest caution. Investors should closely monitor market conditions and HAL's specific news and announcements before making investment decisions. Further analysis, including examination of financial statements, industry trends, and competitor performance, is recommended for a more informed perspective.
```

In [116]:

from IPython import display
display.Markdown(res.raw)

```markdown
## HAL Stock Analysis: Weekly and Monthly Report Summary

**Data Overview:**

*   **Symbol:** HAL
*   **Last Refreshed:**
    *   Weekly: 2025-02-26
    *   Monthly: 2025-02-28
*   **Time Zone:** US/Eastern

**I. Weekly Performance Analysis:**

*   **Recent Trend (February 2025):**  The stock has shown a slight downward trend in February. Opening at 26.30 on 2025-02-26 and closing at 25.92. It experienced a volatile week with a high of 27.14 and a low of 25.735. Previous weeks in February show similar volatility.
*   **January 2025:**  Started high at 30.20 and closed the month lower at 26.02, indicating a declining trend throughout January.  High trading volumes were observed, especially in the latter half of the month.
*   **December 2024:** HAL experienced considerable fluctuation. It began the month at 31.93 and ended at 26.79. A significant drop occurred mid-month (2024-12-20), with the price falling to 25.97 amid high volume.
*   **Late 2024 (Nov-Oct):**  Showed more stability and growth, rising from 27.42 (2024-11-01) to a peak around 32.57 before declining towards the end of November.

**II. Monthly Performance Analysis:**

*   **February 2025:** HAL's monthly performance shows volatility, with a wide range between high and low prices (27.3850 and 25.1600 respectively), and closes lower than it opens, indicating negative momentum.
*   **January 2025:** Similar to the weekly data, the monthly data shows a declining trend from the start to the end of the month.
*   **2024 Overview:**
    *   **Q4 (Oct-Dec):** Started with a moderate correction in October, followed by a recovery in November, and a decline in December.
    *   **Q3 (Jul-Sep):** Demonstrated a strong recovery, particularly in September, with a substantial increase in price and volume.
    *   **Q2 (Apr-Jun):** Showed a general downtrend from April to June.
    *   **Q1 (Jan-Mar):** Started strong in January but experienced a decline through February and March.

**III. Volume Analysis:**

*   **Weekly:** Volumes tend to be higher during weeks with significant price changes, suggesting increased trading activity during periods of uncertainty or key price levels.
*   **Monthly:** Consistently high trading volumes throughout the year, indicating strong investor interest in HAL.  September 2024 recorded the highest volume, coinciding with a significant price increase.

**IV. Insights & Summary:**

*   **Strengths:**
    *   **High Trading Volume:**  Consistent high trading volume suggests strong liquidity and investor interest, making it easier to enter and exit positions.
    *   **Recovery Potential:** Demonstrated ability to recover from downturns, as seen in September 2024.
*   **Weaknesses:**
    *   **Price Volatility:** Significant price fluctuations on both weekly and monthly scales, which could indicate higher risk.
    *   **Recent Downtrend:** The stock has been on a downtrend in early 2025, which may concern investors.
*   **Overall Assessment:**

HAL exhibits characteristics of a volatile stock with potential for significant gains but also substantial risk. The high trading volume and past recovery trends are positive signs. However, the recent downtrend and price volatility suggest caution. Investors should closely monitor market conditions and HAL's specific news and announcements before making investment decisions. Further analysis, including examination of financial statements, industry trends, and competitor performance, is recommended for a more informed perspective.
```